In [1]:
import os

os.getcwd()


'c:\\Users\\HP\\Documents\\ISE\\ISE2\\MachineLearning2\\Projet_final'

In [2]:

import os
os.chdir("c:\\Users\\HP\\Documents\\ISE\\ISE2\\MachineLearning2\\Projet_final")



In [3]:
#-----------------------------------
# 1. Import des bibliothèques
# ----------------------------
import pandas as pd                   # Manipulation de données tabulaires
import json                          # Pour parser les chaînes JSON (ex: genres)
import re                            # Pour expressions régulières (nettoyage de texte)
import nltk                          # Bibliothèque NLP (lemmatisation, stop-words)
import numpy as np                   # Calcul numérique et manipulation de vecteurs
import joblib                        # Sauvegarde/chargement d’objets (modèles, vecteurs)


from nltk.corpus import stopwords                    # Stop-words anglais
from nltk.stem import WordNetLemmatizer              # Lemmatisation des mots
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF vectorisation
from sklearn.decomposition import TruncatedSVD       # Réduction dimensionnelle (LSA)


# ----------------------------
# 2. Téléchargement des stop-words
# ----------------------------
nltk.download("stopwords")   # Téléchargement des stop-words anglais
nltk.download("wordnet")     # Ressources pour la lemmatisation

# ----------------------------
# 3. Chargement du CSV brut
# ----------------------------
df0 = pd.read_csv("movies_metadata.csv", low_memory=False)

# On ne conserve que les 3 colonnes utiles
df = df0[["original_title", "overview", "genres"]].copy()

# ----------------------------
# 4. Extraction et nettoyage des genres
# ----------------------------
def extract_genre_list(genres_str):
    """
    Transforme la chaîne Python-JSON en liste de noms.
    Exemples d'entrée :
      "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}]"
    Retourne une chaîne "Animation|Comedy" ou "" si vide/mal formée.
    """
    try:
        # 4.1 Remplacement des quotes pour JSON valide
        js = genres_str.replace("'", '"')
        items = json.loads(js)
        # 4.2 Extraction des noms de genres
        names = [g["name"] for g in items if "name" in g]
        return "|".join(names)
    except Exception:
        return ""


# Application de l'extraction
df["genres"] = df["genres"].astype(str).apply(extract_genre_list)

# ----------------------------
# 5. Nettoyage du texte (overview)
# ----------------------------
# Initialisation du lemmatizer et des stop-words anglais
stop_en = set(stopwords.words("english"))
lemm    = WordNetLemmatizer()

def clean_overview(text):
    """
    Nettoyage de l'overview :
    - mise en minuscules
    - suppression de tout ce qui n'est pas alphanumérique
    - tokenisation simple, suppression des stop-words et lemmatisation
    """
    # 5.1 Passage en minuscules et suppression ponctuation
    t = re.sub(r"[^a-z0-9\s]", " ", text.lower())
    # 5.2 Lemmatisation et filtrage
    tokens = [
        lemm.lemmatize(w)
        for w in t.split()
        if w not in stop_en and len(w) > 2
    ]
    return " ".join(tokens)

# Application du nettoyage
df["clean_overview"] = df["overview"].fillna("").apply(clean_overview)

# ----------------------------
# 6. Filtrage des films sans genre
# ----------------------------
# On ne garde que les lignes où 'genres' n'est pas vide
df = df[df["genres"].str.strip() != ""].reset_index(drop=True)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df.head

<bound method NDFrame.head of                     original_title  \
0                        Toy Story   
1                          Jumanji   
2                 Grumpier Old Men   
3                Waiting to Exhale   
4      Father of the Bride Part II   
...                            ...   
43019              Caged Heat 3000   
43020                   Robin Hood   
43021                      رگ خواب   
43022          Siglo ng Pagluluwal   
43023                     Betrayal   

                                                overview  \
0      Led by Woody, Andy's toys live happily in his ...   
1      When siblings Judy and Peter discover an encha...   
2      A family wedding reignites the ancient feud be...   
3      Cheated on, mistreated and stepped on, the wom...   
4      Just when George Banks has recovered from his ...   
...                                                  ...   
43019  It's the year 3000 AD. The world's most danger...   
43020  Yet another version of the

In [5]:

# ----------------------------
# 7. Sauvegarde du DataFrame préparé
# ----------------------------
output_csv = "tmdb_for_lsa.csv"
df[["original_title", "clean_overview", "genres"]].to_csv(
    output_csv, index=False, encoding="utf-8-sig"
)
print(f"✅ Fichier préparé : {output_csv} ({len(df)} films avec genre)")

✅ Fichier préparé : tmdb_for_lsa.csv (43024 films avec genre)


In [6]:
# ----------------------------
# 8. Construction du modèle LSA
# ----------------------------

import os
import logging
import numpy as np
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------------------------------------------------------
# CONFIGURATION DU LOGGING
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# -----------------------------------------------------------------------------
# 8.1 Vérification de l'existence du fichier préparé
# -----------------------------------------------------------------------------
INPUT_CSV    = "tmdb_for_lsa.csv"
VECTORIZER_PATH = "tfidf_vectorizer.joblib"
LSA_MODEL_PATH  = "lsa_model.joblib"
LSA_DATA_PATH   = "lsa_data.npz"

if not os.path.isfile(INPUT_CSV):
    logging.error(f"Fichier introuvable : {INPUT_CSV}")
    raise FileNotFoundError(f"Veuillez générer {INPUT_CSV} avant d'exécuter ce script.")

# -----------------------------------------------------------------------------
# 8.2 Chargement du DataFrame préparé
# -----------------------------------------------------------------------------
import pandas as pd
df = pd.read_csv(INPUT_CSV)
titles           = df["original_title"].values      # liste des titres
clean_overviews  = df["clean_overview"].values      # liste des textes nettoyés

logging.info(f"Chargé {len(df)} films depuis {INPUT_CSV}")




# -----------------------------------------------------------------------------
# 8.3 TF-IDF : vectorisation du corpus
# -----------------------------------------------------------------------------
try:
    # 8.3.1 Remplacer tout NaN par chaîne vide
    # --------------------------------------------------
    # clean_overviews est un numpy array ou une liste de pandas Series
    # On reconstruit une liste 100% string :
    corpus = [doc if isinstance(doc, str) else "" for doc in clean_overviews]

    # 8.3.2 Instanciation du vectorizer TF-IDF
    vectorizer = TfidfVectorizer(
        max_df=0.8,        # ignorer les termes trop fréquents
        min_df=5,          # ignorer les termes trop rares
        norm="l2",         # normalisation L2 des vecteurs
        sublinear_tf=True  # tf = 1 + log(tf)
    )
    # 8.3.3 Apprentissage et transformation
    X_tfidf = vectorizer.fit_transform(corpus)
    logging.info(f"TF-IDF construit : matrice creuse de forme {X_tfidf.shape}")

except Exception as e:
    logging.error("Erreur lors de la vectorisation TF-IDF")
    logging.exception(e)
    raise



# -----------------------------------------------------------------------------
# 8.4 Entraînement du modèle LSA (SVD tronquée)
# -----------------------------------------------------------------------------
try:
    n_components = 300
    lsa = TruncatedSVD(
        n_components=n_components,
        algorithm="randomized",
        n_iter=10,
        random_state=42
    )
    X_lsa = lsa.fit_transform(X_tfidf)
    logging.info(f"LSA entraîné : projection en dimension {n_components}")
except Exception as e:
    logging.error("Erreur lors de l'entraînement du modèle LSA")
    raise

# -----------------------------------------------------------------------------
# 8.5 Sauvegarde des artefacts pour réutilisation
# -----------------------------------------------------------------------------
try:
    joblib.dump(vectorizer, VECTORIZER_PATH)
    logging.info(f"Vectorizer TF-IDF sauvegardé sous {VECTORIZER_PATH}")
    
    joblib.dump(lsa, LSA_MODEL_PATH)
    logging.info(f"Modèle LSA sauvegardé sous {LSA_MODEL_PATH}")
    
    # Sauvegarde des vecteurs et des titres (pour recommandation)
    np.savez(
        LSA_DATA_PATH,
        X_lsa   = X_lsa,
        titles  = titles
    )
    logging.info(f"Données LSA sauvegardées sous {LSA_DATA_PATH}")
except Exception as e:
    logging.error("Erreur lors de la sauvegarde des artefacts")
    raise

# -----------------------------------------------------------------------------
# 8.6 Fonction de recommandation "générale"
# -----------------------------------------------------------------------------
def recommend_general(user_title, top_n=10):
    """
    Donne les top_n films les plus similaires à user_title.
    Même si user_title n'existe pas dans la base, on le vectorise et compare
    dans l'espace LSA des descriptions.
    """
    # 1) Charger les artefacts si besoin
    if not hasattr(recommend_general, "vectorizer"):
        recommend_general.vectorizer = joblib.load(VECTORIZER_PATH)
        recommend_general.lsa        = joblib.load(LSA_MODEL_PATH)
        data = np.load(LSA_DATA_PATH, allow_pickle=True)
        recommend_general.X_lsa      = data["X_lsa"]
        recommend_general.titles     = data["titles"]
        logging.info("Artefacts de recommandation chargés")
    
    vec = recommend_general.vectorizer
    model = recommend_general.lsa
    X_train_lsa = recommend_general.X_lsa
    titles = recommend_general.titles
    
    # 2) Vectorisation du titre utilisateur
    tfidf_ut = vec.transform([user_title])
    lsa_ut   = model.transform(tfidf_ut)
    
    # 3) Calcul des similarités cosinus
    sims = cosine_similarity(lsa_ut, X_train_lsa).flatten()
    
    # 4) Tri des indices par score décroissant
    idxs = np.argsort(sims)[::-1][:top_n]
    
    # 5) Affichage des recommandations
    print(f"\n🎬 Recommandations pour « {user_title} » :")
    for i in idxs:
        print(f" - {titles[i]}  (score : {sims[i]:.3f})")


2025-06-02 20:04:07 INFO Chargé 43024 films depuis tmdb_for_lsa.csv
2025-06-02 20:04:12 INFO TF-IDF construit : matrice creuse de forme (43024, 17981)
2025-06-02 20:04:49 INFO LSA entraîné : projection en dimension 300
2025-06-02 20:04:50 INFO Vectorizer TF-IDF sauvegardé sous tfidf_vectorizer.joblib
2025-06-02 20:04:50 INFO Modèle LSA sauvegardé sous lsa_model.joblib
2025-06-02 20:04:52 INFO Données LSA sauvegardées sous lsa_data.npz


In [ ]:
#### -----------------------------------------------------------------------------
# 8.7 Exemple d'exécution si lancé directement
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # Exemple de test
    recommend_general("Inception")
    recommend_general("A Space Odyssey")
    recommend_general("Titanic")
    recommend_general("Batman")
    recommend_general("Toy story")
    recommend_general("Superman")




2025-06-02 20:04:54 INFO Artefacts de recommandation chargés



🎬 Recommandations pour « Inception » :
 - The N Word  (score : 0.613)
 - Double Take  (score : 0.531)
 - Back Issues: The Hustler Magazine Story  (score : 0.426)
 - Programming The Nation?  (score : 0.422)
 - Silly Little Game  (score : 0.395)
 - Cane Toads: The Conquest  (score : 0.372)
 - Antonio Gaudí  (score : 0.364)
 - Can't You Hear the Wind Howl? The Life & Music of Robert Johnson  (score : 0.364)
 - L'età di Cosimo de Medici  (score : 0.360)
 - Patience (After Sebald)  (score : 0.352)

🎬 Recommandations pour « A Space Odyssey » :
 - The Dream Is Alive  (score : 0.894)
 - Lost in Space  (score : 0.758)
 - Hail Columbia!  (score : 0.749)
 - キャプテンハーロック  (score : 0.713)
 - Manhunt in Space  (score : 0.703)
 - Nukes in Space  (score : 0.689)
 - A Beautiful Planet  (score : 0.661)
 - The Phantom Planet  (score : 0.634)
 - Sergeant Dead Head  (score : 0.631)
 - The Man from Planet X  (score : 0.628)

🎬 Recommandations pour « Titanic » :
 - Raise the Titanic  (score : 0.509)
 - Titani

: 